Read in data and do primitive feature engineering as talked in 02

In [6]:
from pandas import read_csv
import re


# perform for every race
perform = read_csv('./horse/data/performance.txt', sep=':', encoding='utf-8')


# Pla.
# Rule 2
perform['Pla.'] = perform['Pla.'].apply(lambda x: x.replace(' 平頭馬', ''))
# Rule 1
perform = perform[
            perform['Pla.'].str.contains('|'.join([str(i) for i in list(range(10))]))
          ].reset_index(drop=True) \

# Course
perform['Course Type'] = perform['Course Type'].apply(lambda x: x.split(' - ')[0])

# Distance
perform['Distance'] = perform['Distance'].apply(lambda x: (x.split(' - ')[1])[:-1])

# Finish Time
perform['Finish Time'] = perform['Finish Time(min)'].astype(int)*60 \
                              + perform['Finish Time(sec)'].astype(float)

# champion
perform['is_champ'] = perform['Pla.'].apply(lambda x: 1 if x==1 else 0)

# money
perform['Race Money'] = perform['Race Money'].apply(lambda x: x[3:].replace(',', ''))

# Transform Data Type
perform['Pla.'] = perform['Pla.'].astype(int)
perform['Distance'] = perform['Distance'].astype(int)
perform['Race Money'] = perform['Race Money'].astype(int)


perform['race_key'] = perform['Race Date'].astype(str)+ '_' + perform['Race No.'].astype(str)

perform = perform[['race_key', 'Race Date'
                   , 'Dr.', 'Distance', 'Field Going', 'Course Type', 'Race Money'
                   , 'Act. Wt.', 'Declare. Horse Wt.', 'Win Odds', 'Horse', 'Jockey', 'Trainer'
                   , 'is_champ', 'Pla.', 'Finish Time']]

cols = ['_'.join(item.replace('.', '').lower().split(' ')) for item in perform.columns.tolist()]

perform.columns = cols
perform = perform.sort_values(by=['race_key', 'pla'], ascending=True)

perform.head()

,race_key,race_date,dr,distance,field_going,course_type,race_money,act_wt,declare_horse_wt,win_odds,horse,jockey,trainer,is_champ,pla,finish_time
53437,2015/04/22_1,2015/04/22,2,1000,好地至快地,草地,575000,120,1186,7.3,有情風(N139),薛寶力,吳定強,0,1,57.37
53438,2015/04/22_1,2015/04/22,3,1000,好地至快地,草地,575000,132,1022,6.1,樂趣(S150),韋達,霍利時,0,2,57.53
53439,2015/04/22_1,2015/04/22,5,1000,好地至快地,草地,575000,121,1085,48.0,穩佔先機(N359),連達文,苗禮德,0,3,57.78
53440,2015/04/22_1,2015/04/22,7,1000,好地至快地,草地,575000,127,1211,7.8,上浦勇將(P285),田泰安,徐雨石,0,4,57.82
53441,2015/04/22_1,2015/04/22,1,1000,好地至快地,草地,575000,124,1088,14.0,大地王者(L251),黎海榮,李易達,0,5,57.89


In [8]:
perform.field_going.value_counts()

好地       35432
好地至快地    14947
好地至黏地     2264
黏地         899
濕慢地        696
濕快地         88
快地          86
黏地至軟地       12
軟地          12
Name: field_going, dtype: int64

In [9]:
perform.course_type.value_counts()

草地       50671
全天候跑道     3765
Name: course_type, dtype: int64